# Reto 01 - Prompt Engineering

Consejo rápido: Para ver la Tabla de Contenidos de este Notebook en VS Code, en la pestaña "Explorer", expande la sección "Outline".

## 1. Experimentación de Parámetros
Primero configuraremos el Desafío. Carga la clave API y las bibliotecas de Python relevantes utilizando las celdas a continuación.

### 1.1 Preparación
#### 1.1.1 Configuración del Entorno


#### Crea o actualiza el archivo de entorno llamado ".env"
Hemos incluido un archivo de entorno de ejemplo para tu referencia. Puedes encontrar el archivo `.env.sample` en la raiz de este repositorio o en la carpeta `Student/Resources` dentro de la documentación del desafío. Siéntete libre de hacer las modificaciones necesarias y renombra el archivo como `.env`.

**NOTA:** Asegúrate de mantener el nombre de cada modelo igual al nombre de implementación del modelo que configuraste en Azure OpenAI.

**NOTA:** Ten en cuenta que algunos modelos de AOAI podrían estar obsoletos. Deberías crear los que estén disponibles según sea necesario. Para completar este primer desafío, gpt-35-turbo es el único modelo que necesitas.

Para obtener información sobre los modelos actuales, consulta: [Modelos del Servicio Azure OpenAI Service](https://learn.microsoft.com/es-mx/azure/ai-services/openai/concepts/models)

####  Configuración del Entorno de Azure OpenAI

In [2]:
import openai
import os
import json
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv())

True

In [3]:
API_KEY = os.getenv("OPENAI_API_KEY")
assert API_KEY, "ERROR: Azure OpenAI Key is missing"
openai.api_key = API_KEY

RESOURCE_ENDPOINT = os.getenv("OPENAI_API_BASE","").strip()
assert RESOURCE_ENDPOINT, "ERROR: Azure OpenAI Endpoint is missing"
assert "openai.azure.com" in RESOURCE_ENDPOINT.lower(), "ERROR: Azure OpenAI Endpoint should be in the form: \n\n\t<your unique endpoint identifier>.openai.azure.com"

openai.api_base = RESOURCE_ENDPOINT
openai.api_type = os.getenv("OPENAI_API_TYPE")
openai.api_version = os.getenv("OPENAI_API_VERSION")

chat_model=os.getenv("CHAT_MODEL_NAME")


#### 1.1.2 Explicación de Parámetros
Especificamos los hiperparámetros para los modelos de Azure OpenAI dentro de las funciones de ayuda. Los usuarios pueden ajustar los parámetros según sus necesidades.


###### **Temperatura**
La temperatura varía de 0 a 2. Aquí tienes un desglose rápido de cómo funciona:
- Temperatura baja (0 a 0.3): Salidas más enfocadas, coherentes y conservadoras.
- Temperatura media (0.3 a 0.7): Creatividad y coherencia equilibradas.
- Alta temperatura (> 0.7): Altamente creativa y diversa, pero potencialmente menos coherente.

###### **Top_p**
Establece el límite de probabilidad para el muestreo de tokens, afectando la amplitud de opciones que considera la IA. Valores más altos conducen a más aleatoriedad, mientras que valores más bajos resultan en salidas más enfocadas.


El parámetro "top_p" actúa como un filtro que controla cuántas palabras o frases diferentes considera el modelo de lenguaje cuando intenta predecir la siguiente palabra. Si configuras el valor de "top p" a 0.5, el modelo de lenguaje solo considerará las 50 palabras o frases más probables que podrían seguir. Pero si configuras el valor de "top p" a 0.9, el modelo de lenguaje considerará las 90 palabras o frases más probables.

Dado que "top_p" y "temperatura" realizan trabajos similares como hiperparámetros, usualmente solo ajustamos uno de ellos en lugar de ambos al mismo tiempo.

###### **Max_tokens**
Max_tokens determina la longitud máxima del texto generado. Al establecer un límite, puedes controlar cuánto texto devolverá el modelo LLM, asegurando que no proporcione una respuesta demasiado larga.

###### **Frequency_penalty**
La penalización de frecuencia asegura que el texto generado sea variado al dar una penalización a los tokens que ya se han utilizado en la respuesta.

Varía de -2.0 a 2.0, con valores más altos resultando en una salida más diversa.

Los valores positivos penalizan nuevos tokens basados en su frecuencia existente en el texto hasta el momento, disminuyendo la probabilidad del modelo de repetir la misma línea textualmente.

#### 1.1.3 Función de Ayuda
A lo largo de este desafío, utilizaremos el modelo `gpt-3.5-turbo` de Azure OpenAI y el [punto final de conexión (endpoint) de chat completions](https://platform.openai.com/docs/guides/chat). 

Esta función de ayuda facilitará el uso de prompts y la visualización de las salidas generadas.

**get_chat_completion** ayuda a crear la respuesta de Azure OpenAI utilizando el modelo de chat de tu elección.

**get_completion_from_messages** ayuda a crear la respuesta de Azure OpenAI utilizando el modelo de chat de tu elección, habilitando el historial de chat.


In [4]:
def get_chat_completion(prompt, model=chat_model):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        engine=model,
        messages=messages,
        max_tokens = 200,
        temperature=1,
        top_p = 1.0
    )
    return response.choices[0].message["content"]

In [5]:
def get_completion_from_messages(messages, model=chat_model, temperature=0):
    response = openai.ChatCompletion.create(
        engine=model,
        messages=messages,
        temperature=temperature # this is the degree of randomness of the model's output
    )

    return response.choices[0].message["content"]



#### Prueba las funciones auxiliares

In [6]:
messages =  f"""
Cuentame una broma.
"""
response = get_chat_completion(messages)
print(response)

¿Por qué los pájaros no usan Facebook?

Porque ya tienen Twitter.


In [7]:
messages =  [      
    {'role':'user', 'content':'tell me a joke'},   
    {'role':'assistant', 'content':'Why did the chicken cross the road'},   
    {'role':'user', 'content':'I don\'t know'}
]
response = get_completion_from_messages(messages, temperature=1)
print(response)

To get to the other side!


#### Tareas para Estudiantes (preguntas abiertas):
1. Crea una función de finalización de texto (completions) para una temperatura muy alta, entre (0,1).
2. Crea una función de finalización de texto (completions) para un valor muy pequeño de max_tokens a fin de obtener respuestas más cortas.
3. Crea funciones de finalización (completions) con 2 valores diferentes de penalización por diversidad entre (0,2).

Prueba las funciones de finalización (completions) que has creado y compara los resultados obtenidos con las pruebas anteriores.

In [8]:
# Prueba aquí la función de completación para una temperatura muy alta entre (0,1)
def get_high_temperature_completion(prompt, model=chat_model):
    """
    Genera una finalización de texto con alta temperatura para respuestas más creativas.

    Args:
        prompt (str): El texto inicial para que el modelo genere una respuesta.
        model (str): El modelo de Azure OpenAI a usar. Por defecto, gpt-3.5-turbo.

    Returns:
        str: Respuesta generada por el modelo.
    """
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        engine=model,
        messages=messages,
        max_tokens=200,  # Longitud máxima de la respuesta
        temperature=1.0,  # Alta aleatoriedad para respuestas creativas
        top_p=1.0         # Considerar todas las palabras más probables
    )
    return response.choices[0].message["content"]

## Probamos la función ##
messages = """
Cuéntame una broma.
"""
response = get_high_temperature_completion(messages)
print("Respuesta del modelo:")
print(response)


Respuesta del modelo:
¿Por qué los pájaros no usan Facebook?

Porque ya tienen Twitter.


In [9]:
# Función de completación para un valor max_tokens muy bajo para obtener respuestas más cortas
def get_short_completion(prompt, model=chat_model):
    """
    Genera una finalización de texto con un valor bajo de max_tokens para respuestas cortas.

    Args:
        prompt (str): El texto inicial para que el modelo genere una respuesta.
        model (str): El modelo de Azure OpenAI a usar. Por defecto, gpt-3.5-turbo.

    Returns:
        str: Respuesta generada por el modelo.
    """
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        engine=model,
        messages=messages,
        max_tokens=20,  # Respuestas cortas con máximo 20 tokens
        temperature=0.7,  # Creatividad moderada
        top_p=1.0        # Considerar todas las palabras más probables
    )
    return response.choices[0].message["content"]


## Probamos la función ##
prompt = """
¿Cuál es la capital de Francia?
"""
response = get_short_completion(prompt)
print("Respuesta del modelo:")
print(response)


Respuesta del modelo:
La capital de Francia es París.


In [10]:
# Prueba aquí las funciones de completación con 2 valores diferentes de penalización por diversidad entre (0,2).
def get_low_diversity_completion(prompt, model=chat_model):
    """
    Genera una finalización de texto con baja penalización de diversidad.

    Args:
        prompt (str): El texto inicial para que el modelo genere una respuesta.
        model (str): El modelo de Azure OpenAI a usar. Por defecto, gpt-3.5-turbo.

    Returns:
        str: Respuesta generada por el modelo.
    """
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        engine=model,
        messages=messages,
        max_tokens=100,
        temperature=0.7,
        top_p=1.0,
        frequency_penalty=0.2  # Penalización baja para menor diversidad
    )
    return response.choices[0].message["content"]

def get_high_diversity_completion(prompt, model=chat_model):
    """
    Genera una finalización de texto con alta penalización de diversidad.

    Args:
        prompt (str): El texto inicial para que el modelo genere una respuesta.
        model (str): El modelo de Azure OpenAI a usar. Por defecto, gpt-3.5-turbo.

    Returns:
        str: Respuesta generada por el modelo.
    """
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        engine=model,
        messages=messages,
        max_tokens=100,
        temperature=0.7,
        top_p=1.0,
        frequency_penalty=2.0  # Penalización alta para mayor diversidad
    )
    return response.choices[0].message["content"]

## Probamos las funciones ##
prompt = """
Escribe una lista de razones para aprender programación.
"""
response_low_diversity = get_low_diversity_completion(prompt)
print("Respuesta con baja penalización por diversidad:")
print(response_low_diversity)

response_high_diversity = get_high_diversity_completion(prompt)
print("\nRespuesta con alta penalización por diversidad:")
print(response_high_diversity)


Respuesta con baja penalización por diversidad:
1. Desarrollar habilidades de resolución de problemas
2. Mejorar la lógica y la capacidad de pensamiento crítico
3. Ampliar las oportunidades de empleo
4. Crear aplicaciones y programas personalizados
5. Automatizar tareas repetitivas
6. Potenciar la creatividad
7. Comprender mejor el funcionamiento de la tecnología
8. Mejorar la colaboración y trabajo en

Respuesta con alta penalización por diversidad:
1. Desarrollar habilidades de resolución de problemas.
2. Mejorar la lógica y el pensamiento crítico.
3. Ampliar las oportunidades laborales y profesionales.
4. Crear aplicaciones y software útiles para la sociedad.
5. Automatizar tareas repetitivas para ahorrar tiempo y esfuerzo.
6. Entender mejor cómo funcionan las tecnologías que utilizamos a diario.
7. F


### 1.2 Ingeniería de Mensajes del Sistema
Los usuarios pueden lograr que los modelos respondan en el tono deseado ajustando el mensaje del sistema.

#### 1.2.1 Cambio de Tono

In [13]:
messages =  [      
    {'role':'assistant', 'content':'How can I help you?'},
    {'role':'user', 'content':'tell me a joke'}
]
response = get_completion_from_messages(messages, temperature=1)
print(response)

Sure, here's one for you:

Why did the scarecrow win an award?
Because he was outstanding in his field!


In [14]:
messages =  [  
    {'role':'system', 'content':'You are an assistant that speaks like Shakespeare.'},    
    {'role':'assistant', 'content':'How can I help you?'},
    {'role':'user', 'content':'tell me a joke'}
]
response = get_completion_from_messages(messages, temperature=1)
print(response)

Of course, kind sir! Here's a jest for thee:

Why did the cookie go to the doctor?

Because it was feeling crumbly! Haha, what a merry jest indeed!


#### Tarea para Estudiantes (pregunta abierta):
Haz que el asistente cuente un chiste con el tono de tu personaje favorito editando el mensaje del sistema.


In [15]:
# Haz que el asistente cuente un chiste con el tono de tu personaje favorito editando el mensaje del sistema.
messages = [
    {'role': 'system', 'content': 'Eres Yoda, un sabio Maestro Jedi de Star Wars, que habla con estructuras de oraciones invertidas.'},
    {'role': 'user', 'content': 'Cuéntame un chiste.'}
]

response = get_completion_from_messages(messages, temperature=1)
print(response)


Chiste te contaré. ¡Por qué Yoda cruza la calle? Para llegar al otro lado, no, no, para estar.


#### 1.2.2 Recordar el Nombre de la Empresa

In [16]:
messages =  [  
{'role':'system', 'content':'You are friendly chatbot.'},  
{'role':'user', 'content':'Hi, my name is Mel.'},
{'role':'assistant', 'content': "Hi! It's nice to meet you. \
Is there anything I can help you with today?"},  
{'role':'user', 'content':'Yes, can you remind me which company I work for?'}  ]
response = get_completion_from_messages(messages, temperature=1)
print(response)

I'm sorry, but I don't have that information. Could you please tell me the name of the company you work for?


#### Tarea para Estudiantes:
Haz que el asistente ofrezca un recordatorio confiable del nombre de la empresa proporcionando contexto en el mensaje del sistema.

In [17]:
# Haz que el asistente ofrezca un recordatorio confiable del nombre de la empresa proporcionando contexto en el mensaje del sistema.
messages = [
    {'role': 'system', 'content': 'Eres un chatbot amigable y servicial. Sabes que el usuario, Mel, trabaja para OpenAI, y puedes usar esta información para ayudar con recordatorios y otras tareas.'},
    {'role': 'user', 'content': 'Hola, me llamo Mel.'},
    {'role': 'assistant', 'content': '¡Hola! Es un gusto conocerte. ¿Hay algo en lo que pueda ayudarte hoy?'},
    {'role': 'user', 'content': 'Sí, ¿puedes recordarme en qué empresa trabajo?'}
]

response = get_completion_from_messages(messages, temperature=1)
print(response)


¡Claro, Mel! Trabajas para OpenAI. ¿Hay algo más en lo que pueda ayudarte?


## 2. Principios de Prompting Iterativo
Es una buena práctica realizar un prompting iterativo para que el modelo pueda generar la respuesta más adecuada según las especificaciones del usuario.
- **Principio 1: Escribe instrucciones claras y específicas**
- **Principio 2: Dale tiempo al modelo para "pensar"**



### 2.1 Escribe instrucciones claras y específicas

#### Táctica 1: Delimitadores

#### Tarea para Estudiantes:
Usa delimitadores para indicar claramente las partes distintas de la entrada.

Los delimitadores pueden ser cualquier cosa como: ````, """, < >, `<tag> </tag>`, `:`

In [18]:
text = f"""
    The 2020 Summer Olympics, officially the Games of the XXXII Olympiad and also known as Tokyo 2020, was an international multi-sport event held from 23 July to 8 August 2021 in Tokyo, Japan, with some preliminary events that began on 21 July 2021. Tokyo was selected as the host city during the 125th IOC Session in Buenos Aires, Argentina, on 7 September 2013.Originally scheduled to take place from 24 July to 9 August 2020, the event was postponed to 2021 on 24 March 2020 due to the global COVID-19 pandemic, the first such instance in the history of the Olympic Games (previous games had been cancelled but not rescheduled). However, the event retained the Tokyo 2020 branding for marketing purposes. It was largely held behind closed doors with no public spectators permitted due to the declaration of a state of emergency in the Greater Tokyo Area in response to the pandemic, the first and only Olympic Games to be held without official spectators. The Games were the most expensive ever, with total spending of over $20 billion.The Games were the fourth Olympic Games to be held in Japan, following the 1964 Summer Olympics (Tokyo), 1972 Winter Olympics (Sapporo), and 1998 Winter Olympics (Nagano). Tokyo became the first city in Asia to hold the Summer Olympic Games twice. The 2020 Games were the second of three consecutive Olympics to be held in East Asia, following the 2018 Winter Olympics in Pyeongchang, South Korea and preceding the 2022 Winter Olympics in Beijing, China. Due to the one-year postponement, Tokyo 2020 was the first and only Olympic Games to have been held in an odd-numbered year and the first Summer Olympics since 1900 to be held in a non-leap year.\nNew events were introduced in existing sports, including 3x3 basketball, freestyle BMX and mixed gender team events in a number of existing sports, as well as the return of madison cycling for men and an introduction of the same event for women. New IOC policies also allowed the host organizing committee to add new sports to the Olympic program for just one Games. The disciplines added by the Japanese Olympic Committee were baseball and softball, karate, sport climbing, surfing and skateboarding, the last four of which made their Olympic debuts, and the last three of which will remain on the Olympic program.The United States topped the medal count by both total golds (39) and total medals (113), with China finishing second by both respects (38 and 89). Host nation Japan finished third, setting a record for the most gold medals and total medals ever won by their delegation at an Olympic Games with 27 and 58. Great Britain finished fourth, with a total of 22 gold and 64 medals. The Russian delegation competing as the ROC finished fifth with 20 gold medals and third in the overall medal count, with 71 medals. Bermuda, the Philippines and Qatar won their first-ever Olympic gold medals. Burkina Faso, San Marino and Turkmenistan also won their first-ever Olympic medals.'
"""

In [19]:
# Usa delimitadores para indicar claramente las partes distintas de la entrada y pide al modelo que resuma el texto.
prompt = f"""
El siguiente texto está delimitado por triples comillas invertidas. Por favor, genera un resumen conciso del contenido.

Texto:
```{text}```
"""

response = get_completion_from_messages(
    [{'role': 'user', 'content': prompt}],
    temperature=0.7
)

print(response)

Los Juegos Olímpicos de Verano de 2020, oficialmente conocidos como Tokio 2020, se llevaron a cabo en Tokio, Japón, del 23 de julio al 8 de agosto de 2021. Fueron los Juegos más caros de la historia, con un gasto total de más de $20 mil millones. Debido a la pandemia de COVID-19, se llevaron a cabo sin espectadores oficiales, siendo la primera vez en la historia de los Juegos Olímpicos. Estados Unidos lideró el medallero con 39 medallas de oro y 113 medallas en total. Japón estableció un récord al ganar 27 medallas de oro y 58 en total. Nuevos deportes y eventos mixtos fueron introducidos, y Japón añadió béisbol y softbol, karate, escalada deportiva, surf y skateboarding al programa olímpico.


#### Táctica 2: Resumen: especificar el número de palabras, extraer información

#### Texto a resumir

In [20]:
game_review = """
The 2020 Summer Olympics, officially the Games of the XXXII Olympiad and also known as Tokyo 2020, \
was an international multi-sport event held from 23 July to 8 August 2021 in Tokyo, Japan, \
with some preliminary events that began on 21 July 2021. Tokyo was selected as the host city during the 125th IOC Session in Buenos Aires, Argentina, \
on 7 September 2013.Originally scheduled to take place from 24 July to 9 August 2020, the event was postponed to 2021 on 24 March 2020 due to the global COVID-19 pandemic, \
the first such instance in the history of the Olympic Games (previous games had been cancelled but not rescheduled). \
However, the event retained the Tokyo 2020 branding for marketing purposes. \
It was largely held behind closed doors with no public spectators permitted due to the declaration of a state of emergency in the Greater Tokyo Area in response to the pandemic, the first and only Olympic Games to be held without official spectators. \
The Games were the most expensive ever, with total spending of over $20 billion. \
The Games were the fourth Olympic Games to be held in Japan, following the 1964 Summer Olympics (Tokyo), 1972 Winter Olympics (Sapporo), \
and 1998 Winter Olympics (Nagano). Tokyo became the first city in Asia to hold the Summer Olympic Games twice. \
The 2020 Games were the second of three consecutive Olympics to be held in East Asia, following the 2018 Winter Olympics in Pyeongchang, \
South Korea and preceding the 2022 Winter Olympics in Beijing, China. \
Due to the one-year postponement, Tokyo 2020 was the first and only Olympic Games to have been held in an odd-numbered year and the first Summer Olympics since 1900 to be held in a non-leap year.
New events were introduced in existing sports, including 3x3 basketball, \
freestyle BMX and mixed gender team events in a number of existing sports, as well as the return of madison cycling for men and an introduction of the same event for women. \
New IOC policies also allowed the host organizing committee to add new sports to the Olympic program for just one Games. \
The disciplines added by the Japanese Olympic Committee were baseball and softball, karate, \
sport climbing, surfing and skateboarding, the last four of which made their Olympic debuts, and the last three of which will remain on the Olympic program. \
The United States topped the medal count by both total golds (39) and total medals (113), with China finishing second by both respects (38 and 89). \
Host nation Japan finished third, setting a record for the most gold medals and total medals ever won by their delegation at an Olympic Games with 27 and 58. \
Great Britain finished fourth, with a total of 22 gold and 64 medals. \
The Russian delegation competing as the ROC finished fifth with 20 gold medals and third in the overall medal count, with 71 medals. \
Bermuda, the Philippines and Qatar won their first-ever Olympic gold medals. Burkina Faso, San Marino and Turkmenistan also won their first-ever Olympic medals.
"""

#### Tarea para Estudiantes:
Crea un resumen, centrándote en los aspectos especiales de los Juegos Olímpicos de Verano 2020 y con un límite de palabras.

In [21]:
# Crea un resumen, centrándote en los aspectos especiales de los Juegos Olímpicos de Verano 2020 y con un límite de palabras.
prompt = f"""
Por favor, genera un resumen limitado a 100 palabras enfocado en los aspectos especiales de los Juegos Olímpicos de Verano 2020. Aquí tienes el texto original:

Texto:
```{game_review}```
"""

# Solicitud al modelo de OpenAI
response = openai.ChatCompletion.create(
    engine=chat_model,
    messages=[
        {"role": "user", "content": prompt}
    ],
    temperature=0.7,
    max_tokens=200
)

# Imprime el resumen generado
print("Resumen Generado:")
print(response.choices[0].message["content"])

Resumen Generado:
Los Juegos Olímpicos de Verano 2020, celebrados en Tokio en 2021 debido a la pandemia de COVID-19, fueron los primeros en realizarse sin espectadores oficiales. Incluyeron nuevos eventos en deportes existentes y la introducción de deportes como el surf y el skateboard. Japón agregó béisbol, karate, escalada deportiva, surf y skateboard al programa, con algunos convirtiéndose en deportes olímpicos permanentes. Estados Unidos lideró el medallero, seguido de China y Japón. Varios países ganaron sus primeras medallas de oro, destacando el éxito de atletas de naciones menos representadas en los Juegos.


#### Tarea para Estudiantes:
Intenta "extract" en vez de "summarize" ("extraer" en vez de "resumir")

In [22]:
# Intenta "extract" en vez de "summarize" ("extraer" en vez de "resumir")
prompt = f"""
Extrae los aspectos especiales de los Juegos Olímpicos de Verano 2020 del siguiente texto. Limita la respuesta a 100 palabras.

Texto:
```{game_review}```
"""

# Solicitud al modelo de OpenAI
response = openai.ChatCompletion.create(
    engine=chat_model,
    messages=[
        {"role": "user", "content": prompt}
    ],
    temperature=0.7,
    max_tokens=200
)

# Imprime la información extraída
print("Información Extraída:")
print(response.choices[0].message["content"])

Información Extraída:
- Los Juegos Olímpicos de Verano 2020 se llevaron a cabo en 2021 en Tokio, Japón, debido a la pandemia de COVID-19.
- Fueron los primeros Juegos Olímpicos en la historia en ser pospuestos y celebrados sin espectadores oficiales.
- Se introdujeron nuevos eventos en deportes existentes, como baloncesto 3x3, BMX estilo libre y eventos mixtos de equipos.
- Hubo la inclusión de nuevos deportes como béisbol y softbol, karate, escalada deportiva, surf y skateboard.
- Japón estableció un récord al ganar la mayor cantidad de medallas de oro y medallas totales en unos Juegos Olímpicos.


#### Táctica 3: Inferencia: solicitar emociones, sentimientos o temas

#### Tarea para Estudiantes:
Identifica los tipos de emociones y el sentimiento (positivo/negativo) de la siguiente reseña.

Usa un objeto JSON.

In [23]:
review = """
Philip Barker of Inside the Games opined that for many athletes and supporters, \
the tone of the ceremony was dignified and appropriate. Hashimoto stated in a press interview that the flame would "quietly go out", \
which he felt that "It was an apt description of a dignified and low key Ceremony which conveyed a sense of gratitude that the Games had been able to take place at all."\
Dominic Patten of Deadline Hollywood argued that the ceremony was an "uneven mixtape" of contrasts, \
comparing the low-key "celebration of the culture of the Asian power and brow moping acknowledgement of the pandemic" to the jubilant Paris segment, \
as well as clichÃ©-filled speech of Thomas Bach. Alan Tyres of The Daily Telegraph discussed the IOC updated motto as a sign of things to come. \
He stated, "The updated Olympic motto of 'faster, higher, \
stronger â€“ together' fits with how sport is covered and contextualised at this moment in history: \
inclusion, diversity, justice and a duty of care to the athletes must be taken into consideration as much as performance." \
He also discussed the strangeness of the ceremony, as it was performed without a stadium audience.
"""

In [24]:
# Identifica los tipos de emociones y el sentimiento (positivo/negativo) de la siguiente reseña. Usa un objeto JSON
prompt = f"""
Analiza las emociones y el sentimiento (positivo/negativo/neutro) de la siguiente reseña. Devuelve la información como un objeto JSON con claves "emociones" y "sentimiento".

Reseña:
```{review}```
"""

# Solicitud al modelo de OpenAI
response = openai.ChatCompletion.create(
    engine=chat_model,
    messages=[
        {"role": "user", "content": prompt}
    ],
    temperature=0.5,
    max_tokens=200
)

# Imprime el resultado en formato JSON
print("Análisis de emociones y sentimientos:")
print(response.choices[0].message["content"])

Análisis de emociones y sentimientos:
{
    "emociones": {
        "positivas": ["dignified", "appropriate", "gratitude", "jubilant", "inclusion", "diversity", "justice", "duty of care"],
        "negativas": ["uneven", "brow moping", "clichÃ©-filled", "strangeness"],
        "neutras": ["quietly", "low key", "updated"]
    },
    "sentimiento": "positivo"
}


#### Desafío para Estudiantes:
Deduce 3 temas (tópicos) de la historia a continuación.

In [25]:
story = """
The 2020 Summer Olympics, officially the Games of the XXXII Olympiad and also known as Tokyo 2020, \
was an international multi-sport event held from 23 July to 8 August 2021 in Tokyo, Japan, \
with some preliminary events that began on 21 July 2021. Tokyo was selected as the host city during the 125th IOC Session in Buenos Aires, Argentina, \
on 7 September 2013.Originally scheduled to take place from 24 July to 9 August 2020, the event was postponed to 2021 on 24 March 2020 due to the global COVID-19 pandemic, \
the first such instance in the history of the Olympic Games (previous games had been cancelled but not rescheduled). \
However, the event retained the Tokyo 2020 branding for marketing purposes. \
It was largely held behind closed doors with no public spectators permitted due to the declaration of a state of emergency in the Greater Tokyo Area in response to the pandemic, the first and only Olympic Games to be held without official spectators. \
The Games were the most expensive ever, with total spending of over $20 billion. \
The Games were the fourth Olympic Games to be held in Japan, following the 1964 Summer Olympics (Tokyo), 1972 Winter Olympics (Sapporo), \
and 1998 Winter Olympics (Nagano). Tokyo became the first city in Asia to hold the Summer Olympic Games twice. \
The 2020 Games were the second of three consecutive Olympics to be held in East Asia, following the 2018 Winter Olympics in Pyeongchang, \
South Korea and preceding the 2022 Winter Olympics in Beijing, China. \
Due to the one-year postponement, Tokyo 2020 was the first and only Olympic Games to have been held in an odd-numbered year and the first Summer Olympics since 1900 to be held in a non-leap year.
New events were introduced in existing sports, including 3x3 basketball, \
freestyle BMX and mixed gender team events in a number of existing sports, as well as the return of madison cycling for men and an introduction of the same event for women. \
New IOC policies also allowed the host organizing committee to add new sports to the Olympic program for just one Games. \
The disciplines added by the Japanese Olympic Committee were baseball and softball, karate, \
sport climbing, surfing and skateboarding, the last four of which made their Olympic debuts, and the last three of which will remain on the Olympic program. \
The United States topped the medal count by both total golds (39) and total medals (113), with China finishing second by both respects (38 and 89). \
Host nation Japan finished third, setting a record for the most gold medals and total medals ever won by their delegation at an Olympic Games with 27 and 58. \
Great Britain finished fourth, with a total of 22 gold and 64 medals. \
The Russian delegation competing as the ROC finished fifth with 20 gold medals and third in the overall medal count, with 71 medals. \
Bermuda, the Philippines and Qatar won their first-ever Olympic gold medals. Burkina Faso, San Marino and Turkmenistan also won their first-ever Olympic medals.
"""

In [26]:
# Deduce 3 temas (tópicos) de la historia a continuación.
prompt = f"""
Analyze the following story and identify 3 key topics. Return the topics as a list of concise phrases.

Story:
```{story}```
"""

# Solicitud al modelo de OpenAI
response = openai.ChatCompletion.create(
    engine=chat_model,  # Cambia al modelo que estés usando
    messages=[
        {"role": "user", "content": prompt}
    ],
    temperature=0.3,
    max_tokens=100
)

# Mostrar los temas deducidos
print("Key Topics:")
print(response.choices[0].message["content"])

Key Topics:
1. Tokyo 2020 Olympics
2. Impact of COVID-19 on the Olympics
3. Medal count and achievements of participating countries


#### Táctica 4: Transformación: especifica el idioma objetivo y el estilo de escritura, y solicita una revisión gramatical.

#### Tarea para Estudiantes:
Traductor Universal

Personas de todo el mundo desean conocer las noticias de los Juegos Olímpicos en su idioma nativo. En este caso, las noticias deben ser traducidas a diferentes idiomas. Traduce cada elemento de noticias a continuación tanto al coreano como al inglés.

In [ ]:
news = [
"Palestine received a universality invitation from FINA to send two top-ranked swimmers (one per gender) in their respective individual events to the Olympics, \
based on the FINA Points System of June 28, 2021.",
"比赛包括两轮：预赛和决赛。预赛成绩最好的8次接力队晋级决赛。必要时使用游泳比赛来打破平局以晋级下一轮。"]

In [29]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chat_models import AzureChatOpenAI  # Asegúrate de usar un modelo de Chat

# Configurar el modelo de AzureChatOpenAI
llm = AzureChatOpenAI(
    deployment_name="gpt-35-turbo",  # Cambia al nombre de tu despliegue
    openai_api_version="2023-03-15-preview",
    temperature=0.5,
)

# Lista de noticias
news = [
    "Palestine received a universality invitation from FINA to send two top-ranked swimmers (one per gender) in their respective individual events to the Olympics, \
based on the FINA Points System of June 28, 2021.",
    "比赛包括两轮：预赛和决赛。预赛成绩最好的8次接力队晋级决赛。必要时使用游泳比赛来打破平局以晋级下一轮。"
]

# Crear el template del prompt
template = """Translate the following text into Korean and English:
    
> Text: {text}

Provide the translations in this format:
- Korean: [Your translation in Korean here]
- English: [Your translation in English here]
"""

# Configurar el template y la cadena
prompt = PromptTemplate(template=template, input_variables=["text"])
translation_chain = LLMChain(llm=llm, prompt=prompt, verbose=False)

# Traducir las noticias
translations = []
for text in news:
    response = translation_chain.run(text=text)
    translations.append(response)

# Imprimir las traducciones
for i, translation in enumerate(translations, start=1):
    print(f"News {i} Translations:\n{translation}\n")


/tmp/ipykernel_5448/842203783.py:6: LangChainDeprecationWarning: The class `AzureChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import AzureChatOpenAI``.
  llm = AzureChatOpenAI(
/tmp/ipykernel_5448/842203783.py:31: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  translation_chain = LLMChain(llm=llm, prompt=prompt, verbose=False)
/tmp/ipykernel_5448/842203783.py:36: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = translation_chain.run(text=text)


News 1 Translations:
- Korean: 팔레스타인은 2021년 6월 28일 FINA 포인트 시스템을 기반으로 올림픽에 각각의 개인 종목에서 상위 랭킹 수영 선수 두 명(성별당 한 명)을 파견할 수 있는 보편적 초청을 받았습니다.
- English: Palestine received a universality invitation from FINA to send two top-ranked swimmers (one per gender) in their respective individual events to the Olympics, based on the FINA Points System of June 28, 2021.

News 2 Translations:
- Korean: 대회는 예선전과 결승전 두 라운드로 진행됩니다. 예선전에서 성적이 가장 우수한 8개의 계주팀이 결승전에 진출합니다. 필요한 경우 수영 경기를 이용하여 동점을 깨고 다음 라운드에 진출합니다.
- English: The competition consists of two rounds: preliminary rounds and finals. The top 8 relay teams with the best results in the preliminary rounds advance to the finals. If necessary, swimming races may be used to break ties and advance to the next round.



#### Tarea para Estudiantes:
Transformación de Tono

La escritura puede variar según la audiencia objetivo. ChatGPT puede producir diferentes tonos. Transforma el siguiente mensaje en una carta de negocios.

In [31]:
original_message = "David, it's John! OMG, the Olympic game is so crazy"

In [33]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chat_models import AzureChatOpenAI

# Configurar el modelo de AzureChatOpenAI
llm = AzureChatOpenAI(
    deployment_name="gpt-35-turbo",  # Cambia al nombre de tu despliegue
    openai_api_version="2023-03-15-preview",
    temperature=0.5,
)

# Crear el template del prompt
template = """Transform the following casual message into a formal business letter:
    
> Original Message: {message}

Your response should follow the structure of a professional letter, include a formal salutation, body, and closing.
"""

# Configurar el template y la cadena
prompt = PromptTemplate(template=template, input_variables=["message"])
tone_transformation_chain = LLMChain(llm=llm, prompt=prompt, verbose=False)

# Transformar el mensaje
response = tone_transformation_chain.run(message=original_message)

# Imprimir el resultado
print("Transformed Business Letter:\n")
print(response)


Transformed Business Letter:

Dear David,

I hope this message finds you well. I wanted to bring to your attention the excitement surrounding the Olympic Games. The level of competition and athleticism displayed by the athletes is truly remarkable. 

Thank you for your attention to this matter.

Sincerely,
John


#### Tarea para Estudiantes:
Conversión de Formato

ChatGPT puede traducir entre formatos. El prompt debe describir los formatos de entrada y salida. Convierte los siguientes datos JSON al formato HTML.

In [34]:
data_json = { "The 2020 Summer Olympics Opening Ceremony audience name list" :[ 
    {"name":"Shyam", "email":"shyamjaiswal@gmail.com"},
    {"name":"Bob", "email":"bob32@gmail.com"},
    {"name":"Jai", "email":"jai87@gmail.com"}
]}

In [35]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chat_models import AzureChatOpenAI

# Configurar el modelo de AzureChatOpenAI
llm = AzureChatOpenAI(
    deployment_name="gpt-35-turbo",  # Cambia al nombre de tu despliegue
    openai_api_version="2023-03-15-preview",
    temperature=0.5,
)

# Crear el template del prompt
template = """Convert the following JSON data into an HTML format:
    
JSON Data:
{json_data}

Your output should contain a structured HTML table that represents the data.
"""

# Configurar el template y la cadena
prompt = PromptTemplate(template=template, input_variables=["json_data"])
format_conversion_chain = LLMChain(llm=llm, prompt=prompt, verbose=False)

# Convertir el JSON a HTML
response = format_conversion_chain.run(json_data=data_json)

# Imprimir el resultado en HTML
print("Converted HTML:\n")
print(response)


Converted HTML:

<!DOCTYPE html>
<html>
<head>
    <title>Olympics Opening Ceremony Audience</title>
</head>
<body>

    <h1>2020 Summer Olympics Opening Ceremony Audience</h1>

    <table border="1">
        <tr>
            <th>Name</th>
            <th>Email</th>
        </tr>
        <tr>
            <td>Shyam</td>
            <td>shyamjaiswal@gmail.com</td>
        </tr>
        <tr>
            <td>Bob</td>
            <td>bob32@gmail.com</td>
        </tr>
        <tr>
            <td>Jai</td>
            <td>jai87@gmail.com</td>
        </tr>
    </table>

</body>
</html>


#### Tarea para Estudiantes:
Revisión ortográfica y gramatical del siguiente texto.

Para indicarle al modelo LLM que deseas que revise tu texto, instruye al modelo para que 'revise' o 'revise y corrija'.

In [ ]:
check_text = [ 
  "Alongside the main Emblem blue, the five other colors use in the branding of the 2020 Games is : Kurenai red,   Ai blue,   Sakula pink,   Fuji purple, and   Matsuba green.",
  "The competition have three round: heats, semifinals, and a final."
]

In [ ]:
# Escribe aquí el código para la tarea .

#### Táctica 5: Ampliación: personaliza la respuesta automática

#### Tarea para Estudiantes:
Personaliza la respuesta automática al siguiente correo electrónico de un cliente.

El cliente tuvo un problema al comprar un boleto para los Juegos Olímpicos.

In [36]:
# given the sentiment from the tactic on "inferring",
# and the original customer message, customize the email
sentiment = "negative"

# review for a ticket transaction
review = f"""
I bought the ticket of "Men's 100 metre freestyle swimming" game last week.\
The transaction went through successfully. However, I still have not received the ticket.\
Over one week has passed.\
"""

In [37]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chat_models import AzureChatOpenAI

# Configurar el modelo de AzureChatOpenAI
llm = AzureChatOpenAI(
    deployment_name="gpt-35-turbo",  # Cambia al nombre de tu despliegue
    openai_api_version="2023-03-15-preview",
    temperature=0.5,
)

# Crear el template del prompt
template = """You are a customer service assistant. Write a professional and empathetic email response to the customer based on their review and sentiment.
    
Sentiment: {sentiment}
Customer Review: {review}

Your response should include:
1. Acknowledgment of the issue.
2. Apology for the inconvenience.
3. Assurance that the issue will be resolved.
4. Clear next steps or actions the customer needs to take (if any).
"""

# Configurar el template y la cadena
prompt = PromptTemplate(template=template, input_variables=["sentiment", "review"])
response_chain = LLMChain(llm=llm, prompt=prompt, verbose=False)

# Generar la respuesta personalizada
response = response_chain.run(sentiment=sentiment, review=review)

# Imprimir el resultado
print("Customized Email Response:\n")
print(response)


Customized Email Response:

Subject: Re: Issue with Ticket Purchase for Men's 100 Metre Freestyle Swimming Game

Dear [Customer],

I am writing in response to your recent review regarding the ticket purchase for the "Men's 100 metre freestyle swimming" game. 

I sincerely apologize for the inconvenience you have experienced in not receiving your ticket even though the transaction was successfully processed over a week ago. This is certainly not the level of service we strive to provide to our customers, and I understand how frustrating this situation must be for you.

Please rest assured that we are taking immediate action to resolve this issue for you. I have escalated your case to our ticketing team, who will investigate the matter and ensure that your ticket is delivered to you promptly. 

In the meantime, if you could kindly provide us with your order number or any relevant details, it would greatly assist us in expediting the process of locating your ticket and ensuring its delive

#### Táctica 6: Chatbot: personalizar conversaciones para tareas o comportamientos específicos

#### Tarea para Estudiantes:
Crear una conversación con el chatbot para saber dónde se celebraron los Juegos Olímpicos de Verano de 2020.

In [39]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chat_models import AzureChatOpenAI

# Configurar el modelo de AzureChatOpenAI
llm = AzureChatOpenAI(
    deployment_name="gpt-35-turbo",  # Cambia al nombre de tu despliegue
    openai_api_version="2023-03-15-preview",
    temperature=0.7,  # Permitir respuestas más naturales
)

# Crear el template del prompt
template = """You are a knowledgeable and friendly chatbot specialized in providing information about the Olympic Games.

User: Where were the 2020 Summer Olympics held?
Assistant:"""

# Configurar el prompt y la cadena
prompt = PromptTemplate(template=template, input_variables=["dummy"])  # Se agrega una variable ficticia
chat_chain = LLMChain(llm=llm, prompt=prompt, verbose=False)

# Generar la respuesta del chatbot
response = chat_chain.run({"dummy": ""})  # Proporciona un diccionario con la clave ficticia

# Imprimir la respuesta
print("Chatbot Response:\n")
print(response)


Chatbot Response:

The 2020 Summer Olympics were originally scheduled to be held in Tokyo, Japan. However, due to the COVID-19 pandemic, the Games were postponed and eventually took place in 2021 in Tokyo.


### 2.2 Dale tiempo al modelo para "pensar"

#### Táctica 1: Especificar los pasos necesarios para completar una tarea
A veces puedes ayudar al modelo a "ralentizarse" y ofrecer respuestas más robustas y detalladas especificando los pasos que debe seguir.

Pidamos la salida en varios formatos especificados.

In [40]:
prompt = f"""
Your task is to help a journalist summarize information from the article for publication.

Write a title based on the information provided in the context delimited by triple backticks. 
The title should be short, catchy, and reflective of the article's narrative.

After the title, generate five keywords from the context.

After the keywords, include a table to organize the information. 
The table should have two columns. In the first column is the title.
In the second column include the keywords as a list.

Give the table the title 'Article Publishing Information'.

Format everything as HTML that can be used in a website.
Place the title in a <div> element.

Context: ```{text}

""" 

get_chat_completion(prompt)

'<div>Summary: Relay Competition at the Swimming Championship</div>\n\nKeywords: Relay, Competition, Swimming, Championship, Preliminary\n\n<table>\n  <tr>\n    <td>Article Publishing Information</td>\n    <td>\n      <ul>\n        <li>Relay</li>\n        <li>Competition</li>\n        <li>Swimming</li>\n        <li>Championship</li>\n        <li>Preliminary</li>\n      </ul>\n    </td>\n  </tr>\n</table>'

#### Táctica 2: Instruir al modelo para que desarrolle su propia solución

Existen dos métodos principales que demostraremos en esta sección para lograr que el modelo resuelva un problema en lugar de apresurarse a una conclusión: cadena de pensamiento (chain-of-thought) y encadenamiento (chaining). Estas estrategias pueden llevar a una mayor precisión, detalle y la capacidad de resolver desafíos complejos.


2.1 - Prompting de cadena de pensamiento (Chain-of-thought)
- Pedir al modelo que razone
- Ejemplo de un solo disparo (one-shot)
- Razonamiento del chatbot

2.2 - Encadenamiento (Chaining)

Continuemos trabajando con el conjunto de datos de los Juegos Olímpicos.

#### 2.2.1 Prompting de Cadena de Pensamiento (Chain-of-Thought Prompting)

Hagamos un poco de matemáticas. Los modelos de GPT ocasionalmente no resuelven muy bien problemas matemáticos directos, así que guiemos al modelo a través del problema.

Descomponemos las tareas en piezas más pequeñas.

Lee más sobre los métodos y la investigación en el documento técnico: https://github.com/openai/openai-cookbook/blob/main/techniques_to_improve_reliability.md

#### Táctica 1: Puedes comenzar pidiéndole específicamente al modelo que piense paso a paso.

In [41]:
text = f"""
    The 2020 Summer Olympics, officially the Games of the XXXII Olympiad and also known as Tokyo 2020, was an international multi-sport event held from 23 July to 8 August 2021 in Tokyo, Japan, with some preliminary events that began on 21 July 2021. Tokyo was selected as the host city during the 125th IOC Session in Buenos Aires, Argentina, on 7 September 2013.Originally scheduled to take place from 24 July to 9 August 2020, the event was postponed to 2021 on 24 March 2020 due to the global COVID-19 pandemic, the first such instance in the history of the Olympic Games (previous games had been cancelled but not rescheduled). However, the event retained the Tokyo 2020 branding for marketing purposes. It was largely held behind closed doors with no public spectators permitted due to the declaration of a state of emergency in the Greater Tokyo Area in response to the pandemic, the first and only Olympic Games to be held without official spectators. The Games were the most expensive ever, with total spending of over $20 billion.The Games were the fourth Olympic Games to be held in Japan, following the 1964 Summer Olympics (Tokyo), 1972 Winter Olympics (Sapporo), and 1998 Winter Olympics (Nagano). Tokyo became the first city in Asia to hold the Summer Olympic Games twice. The 2020 Games were the second of three consecutive Olympics to be held in East Asia, following the 2018 Winter Olympics in Pyeongchang, South Korea and preceding the 2022 Winter Olympics in Beijing, China. Due to the one-year postponement, Tokyo 2020 was the first and only Olympic Games to have been held in an odd-numbered year and the first Summer Olympics since 1900 to be held in a non-leap year.\nNew events were introduced in existing sports, including 3x3 basketball, freestyle BMX and mixed gender team events in a number of existing sports, as well as the return of madison cycling for men and an introduction of the same event for women. New IOC policies also allowed the host organizing committee to add new sports to the Olympic program for just one Games. The disciplines added by the Japanese Olympic Committee were baseball and softball, karate, sport climbing, surfing and skateboarding, the last four of which made their Olympic debuts, and the last three of which will remain on the Olympic program.The United States topped the medal count by both total golds (39) and total medals (113), with China finishing second by both respects (38 and 89). Host nation Japan finished third, setting a record for the most gold medals and total medals ever won by their delegation at an Olympic Games with 27 and 58. Great Britain finished fourth, with a total of 22 gold and 64 medals. The Russian delegation competing as the ROC finished fifth with 20 gold medals and third in the overall medal count, with 71 medals. Bermuda, the Philippines and Qatar won their first-ever Olympic gold medals. Burkina Faso, San Marino and Turkmenistan also won their first-ever Olympic medals.'
"""

# From Azure documentation
prompt = "Who was the most decorated (maximum medals) individual athlete in the Olympic games that were held at Sydney? Take a step-by-step approach in your response, cite sources and give reasoning before sharing final answer in the below format: ANSWER is: <name>"
get_chat_completion(prompt)

'Step 1: Identify the athlete who won the most medals during the Sydney Olympics. \n\nStep 2: Consult official Olympic sources to verify the number of medals won by this athlete.\n\nStep 3: Compare the number of medals won by this athlete to other competitors to confirm they are the most decorated individual athlete at the Sydney Olympics.\n\nStep 4: Present the final answer in the requested format.\n\nStep 1: The athlete who won the most medals during the Sydney Olympics was Michael Phelps, an American swimmer.\n\nStep 2: According to the official Olympic website (olympics.com), Michael Phelps won a total of 8 medals at the 2000 Sydney Olympics. These included 6 gold medals and 2 bronze medals.\n\nStep 3: After reviewing the medal count of all athletes at the Sydney Olympics, it is confirmed that Michael Phelps won the most medals, making him the most decorated individual athlete at the Sydney Olympics.\n\nFinal Answer: Michael Phelps'

In [42]:
# Another example
prompt = f"""
What is the largest time zone difference is between the top two countries who 
won the most gold medals in the 2020 Tokyo olympics?

Use the context below and think aloud as you solve the problem, step-by-step.

Context: {text}
"""
get_chat_completion(prompt)

'To find the largest time zone difference between the top two countries who won the most gold medals in the 2020 Tokyo Olympics, we need to identify the top two countries and determine their respective time zones.\n\nAccording to the context provided, the United States won the most gold medals with 39, while China finished second with 38 gold medals. \n\nThe continental United States spans multiple time zones, with the Eastern Time Zone (UTC-4 to UTC-5), Central Time Zone (UTC-5 to UTC-6), Mountain Time Zone (UTC-6 to UTC-7), and Pacific Time Zone (UTC-7 to UTC-8) being the primary ones. China operates on a single time zone, China Standard Time (UTC+8).\n\nTherefore, the largest time zone difference between the United States and China would involve comparing the Pacific Time Zone in the United States (UTC-7 to UTC-8) with China Standard Time (UTC+8). \n\nThe time zone difference between the'

#### Táctica 2: Ejemplo de un solo disparo (One-shot)
Otra táctica común es proporcionar un ejemplo de una consulta y una respuesta ideal. El modelo aprenderá de ese ejemplo y aplicará los patrones a una nueva pregunta.

In [43]:
# Notice how this response may not be ideal, or the most accurate.
prompt = f"""
    The United States has 113 total medals, 39 of which are gold medals. 
    
    Great Britain has 64 medals and 22 gold medals. 
    
    How many more silver and bronze medals does the United States have over Great Britain?
"""

get_chat_completion(prompt)

'The United States has 74 total silver and bronze medals (113 total medals - 39 gold medals). \nGreat Britain has 42 total silver and bronze medals (64 total medals - 22 gold medals). \n\nTherefore, the United States has 32 more silver and bronze medals than Great Britain.'

In [44]:
# Give the model a one-shot example to solve the question more thoroughly

prompt = f"""
Question: The United States has 113 total medals, 39 of which are gold medals. How many medals are silver or bronze?
Answer:
[Step 1] - There are three types of medals: gold, silver, and bronze
[Step 2] - We know the gold and total counts of medals, so the number of silver and bronze medals is the difference between the gold (39) and total (113) counts. 
            113-39=74, so there are 74 silver and bronze medals combined. The answer is 74.

===

Answer the following question using similar steps above.

Question: China has 89 total medals, 38 of which are gold medals. How many silver and bronze medals do they have?
Answer:
"""

get_chat_completion(prompt)

'[Step 1] - There are three types of medals: gold, silver, and bronze\n[Step 2] - We know the gold and total counts of medals, so the number of silver and bronze medals is the difference between the gold (38) and total (89) counts. \n            89-38=51, so there are 51 silver and bronze medals combined. The answer is 51.'

#### Táctica 3: Razonamiento de cadena de pensamiento del chatbot (chain-of-thought reasoning)
Puedes incorporar lógica utilizando variables para que el chatbot pueda demostrar de manera dinámica formas específicas de pensar sobre un problema.

**Para ingresar la información, aparecerá un cuadro de texto en la parte superior de la pantalla. Deberías ver una ventana emergente. Escribe 'quit' si quieres salir.**

In [45]:
# Ask the bot to help you make a decision such as deciding whether to take a job or choose between restaurants.
# If the model does not immediately respond to your query, wait 1-5 seconds and retype it.
# If it is not allowing you to give an input, restart the kernel in the navigation bar.
# Type "quit" to end the session

context = '''
        You are a decision bot. Your job is to help come to decision by asking a series of questions one at a time and coming to a reasonable decision based on the information provided.

        You will use the following format to help create the series of questions.

        Template: 
        [Problem/Scenario/Question]: [Provide a brief description of the problem, scenario, or question.]

        Chain of thought:

        [Step 1]: Identify the [key element/variable] in the [problem/scenario/question].
        [Step 2]: Understand the [relationship/connection] between [element A] and [element B].
        [Step 3]: [Analyze/Evaluate/Consider] the [context/implication] of the [relationship/connection] between [element A] and [element B].
        [Step 4]: [Conclude/Decide/Determine] the [outcome/solution] based on the [analysis/evaluation/consideration] of [element A], [element B], and their [relationship/connection].
        [Answer/Conclusion/Recommendation]: [Provide a coherent and logical response based on the chain of thought.]

        You will guide the user though a series of questions one at a time. The first question is broad, and they subsequent questions become more specific. 

        Begin by introducing yourself and asking the first question (step 1) only and nothing else, in simple and easy way.
        '''

conversation=[{"role": "system", "content": context}]

while(True):
    print(len(conversation))
    if len(conversation) == 1:
        response = get_completion_from_messages(conversation)
        conversation.append({"role": "assistant", "content": response})
        print("\n" + response + "\n")
    
    user_input = input('Enter your response: ')
    if user_input.lower() == "quit":
        break 
    conversation.append({"role": "user", "content": user_input})
    
    response = get_completion_from_messages(conversation)
    conversation.append({"role": "assistant", "content": response})
    print("\n" + response + "\n")


1

Hello! How can I assist you today?

[Problem/Scenario/Question]: Are you trying to decide between two different job offers?


Great! Let's start by considering the key elements in your decision.

[Step 1]: Identify the key elements in your decision between the two job offers.

4

When comparing the two job offers, let's first consider the key elements that are important to you in making this decision. 

[Step 1]: Identify the key elements that are important to you in choosing between the two job offers.

6

To better understand your decision-making process, let's delve into the relationship between the key elements you've identified and how they align with each job offer.

[Step 2]: Understand how the key elements you've identified relate to each of the job offers.

8

To make a well-informed decision, it's crucial to analyze the context and implications of how the key elements you've identified align with each job offer.

[Step 3]: Analyze the context and implications of how the ke

KeyboardInterrupt: Interrupted by user

#### 2.2.2 - Encadenamiento / Chaining
Similar a algunos ejemplos anteriores, puedes usar las salidas del modelo de consultas anteriores en otras consultas. Más adelante en el Hack te mostraremos cómo hacer esto a escala.

In [47]:
# Extract medal counts for each country from the news article
# Write an article about the summarized information
# Provide a title for the summary

text = f"""
    The 2020 Summer Olympics, officially the Games of the XXXII Olympiad and also known as Tokyo 2020, was an international multi-sport event held from 23 July to 8 August 2021 in Tokyo, Japan, with some preliminary events that began on 21 July 2021. Tokyo was selected as the host city during the 125th IOC Session in Buenos Aires, Argentina, on 7 September 2013.Originally scheduled to take place from 24 July to 9 August 2020, the event was postponed to 2021 on 24 March 2020 due to the global COVID-19 pandemic, the first such instance in the history of the Olympic Games (previous games had been cancelled but not rescheduled). However, the event retained the Tokyo 2020 branding for marketing purposes. It was largely held behind closed doors with no public spectators permitted due to the declaration of a state of emergency in the Greater Tokyo Area in response to the pandemic, the first and only Olympic Games to be held without official spectators. The Games were the most expensive ever, with total spending of over $20 billion.The Games were the fourth Olympic Games to be held in Japan, following the 1964 Summer Olympics (Tokyo), 1972 Winter Olympics (Sapporo), and 1998 Winter Olympics (Nagano). Tokyo became the first city in Asia to hold the Summer Olympic Games twice. The 2020 Games were the second of three consecutive Olympics to be held in East Asia, following the 2018 Winter Olympics in Pyeongchang, South Korea and preceding the 2022 Winter Olympics in Beijing, China. Due to the one-year postponement, Tokyo 2020 was the first and only Olympic Games to have been held in an odd-numbered year and the first Summer Olympics since 1900 to be held in a non-leap year.\nNew events were introduced in existing sports, including 3x3 basketball, freestyle BMX and mixed gender team events in a number of existing sports, as well as the return of madison cycling for men and an introduction of the same event for women. New IOC policies also allowed the host organizing committee to add new sports to the Olympic program for just one Games. The disciplines added by the Japanese Olympic Committee were baseball and softball, karate, sport climbing, surfing and skateboarding, the last four of which made their Olympic debuts, and the last three of which will remain on the Olympic program.The United States topped the medal count by both total golds (39) and total medals (113), with China finishing second by both respects (38 and 89). Host nation Japan finished third, setting a record for the most gold medals and total medals ever won by their delegation at an Olympic Games with 27 and 58. Great Britain finished fourth, with a total of 22 gold and 64 medals. The Russian delegation competing as the ROC finished fifth with 20 gold medals and third in the overall medal count, with 71 medals. Bermuda, the Philippines and Qatar won their first-ever Olympic gold medals. Burkina Faso, San Marino and Turkmenistan also won their first-ever Olympic medals.'
"""

prompt = f"""
    Based on the context below, write a JSON object that contains the number of gold and total medals for each country.
    Context: {text}
"""
num_medals_dict = get_chat_completion(prompt)

prompt = f"""
    Write a brief article about the winners and losers of the Olympics based on medal count:
    {num_medals_dict}
"""
summary = get_chat_completion(prompt)
print(summary)

prompt = f"""
    Give the summary a title in 5 words:
    {summary}
"""
title = get_chat_completion(prompt)
print(title)

"gold": 1,
            "total": 2
        },
        {
            "name": "Greece",
            "gold": 1,
            "total": 6
        }
    ]
}

The Tokyo 2020 Olympics have come to an end, and as always, the medal count tells a story of triumph and defeat for countries around the world. The United States emerged as the clear winners, topping the leaderboard with a total of 113 medals, including an impressive 39 gold medals. The US athletes showcased their talent across a wide range of sports, securing their dominance in the games.

China closely followed the United States, finishing with 89 total medals, 38 of which were gold. The Chinese team put on a strong performance in various events, proving themselves as formidable competitors in the global sporting arena.

Japan, as the host nation, also had a successful Olympics, with 27 gold medals and a total of 58 medals overall. The Japanese athletes' performance
"USA Dominates Tokyo 2020 Olympics"


#### Tarea para Estudiantes (Encadenamiento / Chaining):

Tu tarea ahora es escribir código que determine el país que ganó la mayor cantidad de medallas de plata y bronce combinadas.

Podemos ver que el modelo tiene un rendimiento deficiente al responder la pregunta directamente.

In [48]:
prompt = f"""
    Based on the context, which country had the most silver and bronze medals?
    Context: {text}
"""

get_chat_completion(prompt)

'According to the context, the United States had the most silver medals and bronze medals at the 2020 Summer Olympics.'

Escribe prompts en la celda a continuación que ayudarán al modelo a responder la pregunta desglosando las tareas en diferentes pasos. Asegúrate de que responda solo con la información que se le proporcionó. Este concepto de fundamentación se presentará con más detalle en el Reto 3.

Deberías poder lograr que el modelo responda la pregunta en 2-3 pasos.

In [49]:
# Paso 1: Extraer los datos de medallas en formato JSON
prompt_step1 = f"""
Based on the context below, extract the number of gold, silver, and bronze medals for each country as a JSON object. 
Ensure the JSON includes fields for 'gold', 'silver', and 'bronze' for each country.

Context: {text}
"""
medals_json = get_chat_completion(prompt_step1)
print("Step 1: Extracted Medal Data (JSON):")
print(medals_json)

# Paso 2: Calcular las medallas de plata y bronce combinadas
prompt_step2 = f"""
Based on the following JSON data, calculate the combined total of silver and bronze medals for each country. 
Return the results as a JSON object with each country as the key and their combined silver and bronze medal count as the value.

Data: {medals_json}
"""
silver_bronze_totals = get_chat_completion(prompt_step2)
print("\nStep 2: Combined Silver and Bronze Medals:")
print(silver_bronze_totals)

# Paso 3: Determinar el país con el mayor número de medallas de plata y bronce combinadas
prompt_step3 = f"""
From the following JSON data, determine which country has the highest combined total of silver and bronze medals.
Return the result as a single sentence.

Data: {silver_bronze_totals}
"""
highest_silver_bronze = get_chat_completion(prompt_step3)
print("\nStep 3: Country with Most Silver and Bronze Medals:")
print(highest_silver_bronze)


Step 1: Extracted Medal Data (JSON):
{
    "United States": {
        "gold": 39,
        "silver": 41,
        "bronze": 33
    },
    "China": {
        "gold": 38,
        "silver": 32,
        "bronze": 19
    },
    "Japan": {
        "gold": 27,
        "silver": 14,
        "bronze": 17
    },
    "Great Britain": {
        "gold": 22,
        "silver": 21,
        "bronze": 21
    },
    "ROC": {
        "gold": 20,
        "silver": 28,
        "bronze": 23
    },
    "Bermuda": {
        "gold": 1,
        "silver": 0,
        "bronze": 2
    },
    "Philippines": {
        "gold": 1,
        "silver": 2,
       

Step 2: Combined Silver and Bronze Medals:
    "bronze": 1
}

Results: {
    "United States": 74,
    "China": 51,
    "Japan": 31,
    "Great Britain": 42,
    "ROC": 51,
    "Bermuda": 2,
    "Philippines": 3
}

Step 3: Country with Most Silver and Bronze Medals:
The United States has the highest combined total of silver and bronze medals with 75 in total.


## Conclusión

En este primer desafío, cubrimos cómo interactuar con Azure OpenAI para diferentes objetivos. Esperamos que hayas podido ver la versatilidad dinámica de los modelos y cómo pueden usarse para resolver una variedad de problemas utilizando diferentes técnicas.

Le dimos a la API piezas cortas de texto usando variables fijas. En el próximo conjunto de desafíos, verás cómo usar la API con conjuntos de datos más grandes.